In [1]:
import pandas as pd
import numpy as num

In [3]:
df=pd.read_csv('Fraud_check.csv')
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [4]:
df.insert(6,'new'," ")

for i in range(0,len(df['Taxable.Income']),1):
    if df['Taxable.Income'].iloc[i] <= 30000:
        df['new'].iloc[i]='Risky'
    else:
        df['new'].iloc[i]='Good'

D:\anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,new
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good
...,...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES,Good
596,YES,Divorced,69967,55369,2,YES,Good
597,NO,Divorced,47334,154058,0,YES,Good
598,YES,Married,98592,180083,17,NO,Good


In [7]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Undergrad']= label_encoder.fit_transform(df['Undergrad'])
df['Urban']= label_encoder.fit_transform(df['Urban'])

In [8]:
df.drop(columns=['Taxable.Income','Marital.Status'],inplace=True)

In [9]:
df['new']= label_encoder.fit_transform(df['new'])

In [10]:
df

,Undergrad,City.Population,Work.Experience,Urban,new
0,0,50047,10,1,0
1,1,134075,18,1,0
2,0,160205,30,1,0
3,1,193264,15,1,0
4,0,27533,28,0,0
...,...,...,...,...,...
595,1,39492,7,1,0
596,1,55369,2,1,0
597,0,154058,0,1,0
598,1,180083,17,0,0


In [11]:
df.new.value_counts()

0    476
1    124
Name: new, dtype: int64

In [12]:
x=df.iloc[:,0:4]
y=df['new']

In [14]:
from sklearn.model_selection import train_test_split
# Splitting data into training and testing data set
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size=0.2,random_state=40)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

####  GRID cv

In [16]:
rf=RandomForestClassifier(random_state=42)

In [17]:
param_grid = { 
    'n_estimators': [100,150,200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,4,10,12,15],
    'criterion' :['gini', 'entropy']
}

In [18]:
CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rf.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 10, 12, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 150, 200]})

In [19]:
CV_rf.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'auto',
 'n_estimators': 100}

In [20]:
rf1=RandomForestClassifier(random_state=42, criterion= 'gini',max_depth= 2,max_features= 'auto',n_estimators= 100)

In [24]:
rf1.fit(x_train,y_train)

RandomForestClassifier(max_depth=2, random_state=42)

In [25]:
y_predict = rf1.predict(x_test)

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, y_predict)

0.7666666666666667

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_predict)

array([[92,  0],
       [28,  0]], dtype=int64)

In [28]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87        92
           1       0.00      0.00      0.00        28

    accuracy                           0.77       120
   macro avg       0.38      0.50      0.43       120
weighted avg       0.59      0.77      0.67       120



D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from sklearn import ensemble

# Creating classifier Object
ada = ensemble.AdaBoostClassifier()
#Fitting the classifier to training data
ada.fit(x_train,y_train)

# Making Predictions
ada_pred = ada.predict(x_test)

print("Traing Score:%f"%ada.score(x_train,y_train))
print("Testing Score:%f"%ada.score(x_test,y_test))

Traing Score:0.810417
Testing Score:0.758333
